## Ejercicio 2: Unir diccionarios

In [9]:
from typing import List
from typing import Dict
import json

def unir_diccionarios(a_unir: List[Dict[str,str]]) -> Dict[str,List[str]]:

  out:dict = dict()

  for d in a_unir:
    for key, value in d.items():
        if key not in out.keys():
            out[key] = [value]
        else:
            out[key].append(value)

  return out

In [10]:
unir_diccionarios([{'a': 1, 'b': 2}, {'b': 3, 'c': 4}, {'a': 5}])

{'a': [1, 5], 'b': [2, 3], 'c': [4]}

## Ejercicio 3: Procesar pedidos

In [1]:
from queue import Queue
from typing import List
from typing import Dict
from typing import Union
import json

# ACLARACIÓN: El tipo de "pedidos" debería ser: pedidos: Queue[Dict[str, Union[int, str, Dict[str, int]]]]
# Por no ser soportado por la versión de CMS, usamos simplemente "pedidos: Queue"
def procesamiento_pedidos(pedidos: Queue,
                          stock_productos: Dict[str, int],
                          precios_productos: Dict[str, float]) -> List[Dict[str, Union[int, str, float, Dict[str, int]]]]:
    
    # lista con los pedidos procesados
    res = []

    # proceso los pedidos de a uno
    while (not pedidos.empty()):

        # leo el pedido      
        pedido:dict = pedidos.get()

        id = pedido['id']
        cliente = pedido['cliente']
        productos = pedido['productos'].copy()

        # veo si hay stock, 
        # en tal caso, sumo el precio por la cantidad de productos comprados
        # y descuento del stock

        precio_total:float = 0
        estado:str = "completo" # estado del pedido

        for producto, cantidad in productos.items():

            precio_producto:float = precios_productos[producto]
            cantidad_disponible:int = stock_productos[producto]

            if cantidad_disponible >= cantidad:
                stock_productos[producto] -= cantidad
                precio_total += cantidad * precio_producto
            else:
                estado = "incompleto"       
                stock_productos[producto] = 0
                precio_total += cantidad_disponible * precio_producto
                
                # solo voy a devolver los que habia en stock 
                productos[producto] = cantidad_disponible 

            pass
        
        # armo el pedido procesado
        pedido_procesado:dict() = {
            'id': id,
            'cliente': cliente, 
            'productos': productos,
            'precio_total': precio_total,
            'estado': estado
        }
        res.append(pedido_procesado)

        pass
    
    return res

# Ejemplo input  
# pedidos: [{"id":21,"cliente":"Gabriela", "productos":{"Manzana":2}}, {"id":1,"cliente":"Juan","productos":{"Manzana":2,"Pan":4,"Factura":6}}]
# stock_productos: {"Manzana":10, "Leche":5, "Pan":3, "Factura":0}
# precios_productos: {"Manzana":3.5, "Leche":5.5, "Pan":3.5, "Factura":5

In [11]:
pedidos_ls = [{"id":21,"cliente":"Gabriela", "productos":{"Manzana":2}},
              {"id":1,"cliente":"Juan","productos":{"Manzana":2,"Pan":4,"Factura":6}}]
pedidos = Queue()
for p in pedidos_ls:
    pedidos.put(p)

stock_productos = {"Manzana":10, "Leche":5, "Pan":3, "Factura":0}
precios_productos =  {"Manzana":3.5, "Leche":5.5, "Pan":3.5, "Factura":5}

procesamiento_pedidos(pedidos,stock_productos,precios_productos)

[{'id': 21,
  'cliente': 'Gabriela',
  'productos': {'Manzana': 2},
  'precio_total': 7.0,
  'estado': 'completo'},
 {'id': 1,
  'cliente': 'Juan',
  'productos': {'Manzana': 2, 'Pan': 3, 'Factura': 0},
  'precio_total': 17.5,
  'estado': 'incompleto'}]

## Ejercicio 4: filas del Banco

In [37]:
from queue import Queue

# El tipo de fila debería ser Queue[int], pero la versión de python del CMS no lo soporta.
# Usaremos en su lugar simplemente "Queue"

def avanzarFila(fila: Queue, min: int, verb=False):
    
    # la idea es que estadoCajas[i] = # minutos hasta que se desocupa la iesima caja
    # tiempo[i] = tiempo_entre_clientes - 1 
    # se resta -1 para contar el minuto actual
    
    estadoCajas:dict = {
        "Caja1": {"minutos": 0, "tiempo": 10-1},
        "Caja2": {"minutos": 10, "tiempo": 4-1},
        "Caja3": {"minutos": 10, "tiempo": 4-1, "cliente": ""}
    }

    nuevo_cliente:int = fila.qsize() + 1

    for minuto in range(min+1):
        
        if verb:
            print(minuto)
            for ckey in estadoCajas.keys():
                print(ckey, estadoCajas[ckey])

        # actualizo estadoCajas y busco cajaDisponible
        cajasDisponibles:list = []

        for ckey in estadoCajas.keys():
            tiempo_restante:int = estadoCajas[ckey]["minutos"]

            if tiempo_restante > 0:
                estadoCajas[ckey]["minutos"] -= 1            

            # una caja que se libera al minuto n puede atender desde el minuto n+1
            elif tiempo_restante == 0: 

                # la primer caja en desocuparse en orden ascendente es ocupada
                cajasDisponibles.append(ckey)
                
            # el cliente en Caja3 tarda 3 minutos en volver a la fila,
            # pero Caja3 se libera recien en 4 minutos
            if ckey == "Caja3" and tiempo_restante == 1:
                cliente:int = estadoCajas[ckey]["cliente"]
                if cliente != "":  
                    if verb:    
                        print(f"Vuelve a la fila el cliente {cliente}")
                    fila.put(cliente)
            pass

        hayCajasDisponible:bool = (len(cajasDisponibles) > 0)

        # si hay una caja libre, desencolo de la fila
        if hayCajasDisponible: 

            for cajaAsignada in cajasDisponibles:              

                if fila.empty():
                    break
                  
                cliente:int = fila.get()

                if verb:
                    print(f"cliente {cliente} -> caja {cajaAsignada}")

                estadoCajas[cajaAsignada]["minutos"] += estadoCajas[cajaAsignada]["tiempo"]   

                if cajaAsignada == "Caja3":
                    estadoCajas[cajaAsignada]["cliente"] = cliente 
        
        # agrego una persona a la fila
        # por los tests de la consigna entiendo que esto va al final,
        # es decir acá
        if minuto % 4 == 0:
            if verb:
                print("Nuevo cliente: ", nuevo_cliente)
            fila.put(nuevo_cliente)
            nuevo_cliente += 1

        if verb:
            print()

    pass

    pass


In [39]:
fila = armarCola([1])
avanzarFila(fila, 11, verb=True)
mostrarFila(fila)

0
Caja1 {'minutos': 0, 'tiempo': 9}
Caja2 {'minutos': 10, 'tiempo': 3}
Caja3 {'minutos': 10, 'tiempo': 3, 'cliente': ''}
cliente 1 -> caja Caja1
Nuevo cliente:  2

1
Caja1 {'minutos': 9, 'tiempo': 9}
Caja2 {'minutos': 9, 'tiempo': 3}
Caja3 {'minutos': 9, 'tiempo': 3, 'cliente': ''}

2
Caja1 {'minutos': 8, 'tiempo': 9}
Caja2 {'minutos': 8, 'tiempo': 3}
Caja3 {'minutos': 8, 'tiempo': 3, 'cliente': ''}

3
Caja1 {'minutos': 7, 'tiempo': 9}
Caja2 {'minutos': 7, 'tiempo': 3}
Caja3 {'minutos': 7, 'tiempo': 3, 'cliente': ''}

4
Caja1 {'minutos': 6, 'tiempo': 9}
Caja2 {'minutos': 6, 'tiempo': 3}
Caja3 {'minutos': 6, 'tiempo': 3, 'cliente': ''}
Nuevo cliente:  3

5
Caja1 {'minutos': 5, 'tiempo': 9}
Caja2 {'minutos': 5, 'tiempo': 3}
Caja3 {'minutos': 5, 'tiempo': 3, 'cliente': ''}

6
Caja1 {'minutos': 4, 'tiempo': 9}
Caja2 {'minutos': 4, 'tiempo': 3}
Caja3 {'minutos': 4, 'tiempo': 3, 'cliente': ''}

7
Caja1 {'minutos': 3, 'tiempo': 9}
Caja2 {'minutos': 3, 'tiempo': 3}
Caja3 {'minutos': 3, 'tiempo

[]

In [27]:
def armarCola(ls:list) -> Queue:
    q:Queue = Queue()
    for n in ls:
        q.put(n)
    return q

def mostrarFila(q:Queue) -> list:
    out:list = []
    while not q.empty():
        out.append(q.get())
    return out

In [20]:
def test1():
    
    inicio:list = [1,2,3]
    expected:list = [
                     [1,2,3,4], # 0
                     [2,3,4], # 1
                     [3,4], # 2
                     [4], # 3
                     [4,5], # 4
                     [4,5,2] # 5
                    ]

    for min in range(len(expected)):

        fila:Queue = Queue()
        for n in inicio:
            fila.put(n)
        
        avanzarFila(fila, min)
        
        res:list = []
        for i in range(0, fila.qsize()):
            res.append(fila.get())
        
        if expected[min] != res:
            print(f"Test 1: min: {min} res: {res} expected: {expected[min]}")
            print()
            return False

    return True

def test2():
    
    inicio = []
    expected:list = [
                     [1], # 0 
                     [], # 1
                     [], # 2 
                     [], # 3
                     [2], # 4
                     [], # 5
                     [], # 6
                     [], # 7
                     [3], # 8
                     [], # 9
                     [], # 10
                     [], # 11
                     [4] # 12
                    ] 

    for min in range(len(expected)):

        fila:Queue = Queue()
        for n in inicio:
            fila.put(n)
        
        avanzarFila(fila, min)
        
        res:list = []
        for i in range(0, fila.qsize()):
            res.append(fila.get())
        
        if expected[min] != res:
            print(f"Test 2: min: {min} res: {res} expected: {expected[min]}")
            print()
            return False

    return True

def testSuitConsigna():
    print("Test1: ", test1())
    print("Test2:", test2())

In [17]:
testSuitConsigna()

Test1:  True
Test2: True
